# Homework 2

## Question 1

Write a function that returns the name of a company for a given ticker symbol. 

You should scrape the data from Yahoo! Finance. [Here's an example page](https://finance.yahoo.com/quote/WMT/profile) using Walmart. Start by making your function modify the URL so it works for other ticker symbols. Then, look at the HTML source to find the pattern that you must search for to extract the company name.

I've started the function for you here.

In [32]:
import requests, re

In [343]:
def get_name(ticker):
    # put the URL here
    url = "https://finance.yahoo.com/quote/"
    url = url + ticker
    # download
    req = requests.get(url)
    html = req.text
    try:
        # use a regular expression to find the name
        # (Your code goes here)
        ticker_pattern = re.search("(<meta\s+name=\"keywords\"\s+content=\"[^\&]+)(.*\"/>)",html)
        ticker_pattern = str(ticker_pattern)
        regex = "content=\""+ticker +",([^\&]+[A-Za-z0-9])"
        result = str(re.search(regex,ticker_pattern).groups(1)[0])
        print(result)
    except:    
        # if the regular expression fails, do nothing
        print("Failed to get the company name")

Test your function using these tickers: `AMZN`, `WMT`, `MMM`. Be sure it works as expected.

In [344]:
get_name("AMZN")

 AMAZON COM I


In [345]:
get_name("WMT")

 WALMART INC


In [346]:
get_name("MMM")

 3M COMPANY, M


## Question 2
Write a function that uses the same web page of data to find the address of the company's headquarters.

For example, your function should work as follows:

    In  []: get_address('TSLA')
    Out []: '3500 Deer Creek Road, Palo Alto, CA, 94304'
    
Be sure to test your function with some other stock tickers.

Hint: You'll probably want to capture different parts of the address (street address, city, state, zip) separately and then join them together into a string before returning a value. 

In [681]:
def get_address(ticker):
    # put the URL here
    url = "https://finance.yahoo.com/quote/"
    url = url + ticker
    # download
    req = requests.get(url)
    html = req.text
    try:
        street = str(re.findall('address1":"(.*?)"',html)[0])
        city = re.findall('city":"(.*?)"',html)[0]
        state = re.findall('state":"(.*?)"',html)[0]
        zip = re.findall('zip":"(.*?)"',html)[0]
        return (street+", "+city+", "+state+", "+zip)
    except:
        print("No Address found for the company")
        

In [682]:
get_address("WMT")

'702 SW 8th Street, Bentonville, AR, 72716'

In [683]:
get_address("AMZN")

'410 Terry Avenue North, Seattle, WA, 98109'

In [684]:
get_address("MMM")

'3M Center, St. Paul, MN, 55144'

In [685]:
get_address("TSLA")

'3500 Deer Creek Road, Palo Alto, CA, 94304'

## Question 3

Now I want you to write a *geolocating* function that takes a ticker symbol and returns the latitude and longitude of the firm's headquarters. To do this, we will use Mapquest's API (<b>A</b>pplication <b>P</b>rogramming <b>I</b>nterface), which is a web-based system for providing raw data rather than HTML pages.

Before accessing the API you need to create an account which generates an API Key. The account is free and allows you to make a limited number of queries. To sign up, go [here](https://developer.mapquest.com/user/register) and choose Sign Up. You are assigned a key when you register. It will look something like this: 

<pre>zAWt8Aye6YvhIupZMzctaAfkSkEnaccN</pre>

Once you have created your key, you can query the API using a URL like this:

http://www.mapquestapi.com/geocoding/v1/address?key=API_KEY&location=1309+E+10th+Street%2C+Bloomington%2C+IN%2C+47401

(This link won't work until you replace `API_KEY` with your personal key.)

Notice there is a special formatting to deal with spaces and other punctuation. An easy way to request a web site like this is as follows:

In [643]:
API_KEY = "MI30TLS9nlJyehgSHY2yZK4Ko37WYH5m"
url = 'http://www.mapquestapi.com/geocoding/v1/address'
params = {'key': API_KEY, 'location': '3500 Deer Creek Road, Palo Alto, CA, 94304'}
text = requests.get(url, params=params).text

In [644]:
text

'{"info":{"statuscode":0,"copyright":{"text":"\\u00A9 2019 MapQuest, Inc.","imageUrl":"http://api.mqcdn.com/res/mqlogo.gif","imageAltText":"\\u00A9 2019 MapQuest, Inc."},"messages":[]},"options":{"maxResults":-1,"thumbMaps":true,"ignoreLatLngInput":false},"results":[{"providedLocation":{"location":"3500 Deer Creek Road, Palo Alto, CA, 94304"},"locations":[{"street":"3500 Deer Creek Rd","adminArea6":"","adminArea6Type":"Neighborhood","adminArea5":"Palo Alto","adminArea5Type":"City","adminArea4":"Santa Clara","adminArea4Type":"County","adminArea3":"CA","adminArea3Type":"State","adminArea1":"US","adminArea1Type":"Country","postalCode":"94304-1317","geocodeQualityCode":"P1AAA","geocodeQuality":"POINT","dragPoint":false,"sideOfStreet":"R","linkId":"r7761292|p26337510|n9444498","unknownInput":"","type":"s","latLng":{"lat":37.394738,"lng":-122.149112},"displayLatLng":{"lat":37.394148,"lng":-122.150391},"mapUrl":"http://www.mapquestapi.com/staticmap/v5/map?key=MI30TLS9nlJyehgSHY2yZK4Ko37WYH5m&

In [667]:
re.findall('[^\&=][0-9]+\.[0-9]+',text)

[':37.394738',
 '-122.149112',
 ':37.394148',
 '-122.150391',
 '37.394738',
 '-122.149112']

The response from the server is data from the database stored in JSON format. But you can ignore that it's JSON and just use a regular expression to find the pattern you're looking for.

As an example, your function should return a tuple *exactly* like this:

    In  [2]: geolocate('TSLA')
    Out [2]: (37.394738, -122.149112, '3500 Deer Creek Road, Palo Alto, CA, 94304')
    
That is, it returns a tuple with the latitude, longitude, and address.

The function you write for this question should start by calling the function you wrote in Q2 to get the address. You can then use this with the API to get the location

In [692]:
def geolocate(ticker):
    try:
        API_KEY = "MI30TLS9nlJyehgSHY2yZK4Ko37WYH5m"
        url = 'http://www.mapquestapi.com/geocoding/v1/address'
        location = get_address(ticker)
        params = {'key': API_KEY, 'location':location}
        t = requests.get(url, params=params).text
        regex = re.findall('[^\&=:][0-9]+\.[0-9]+',t)
        latitude =  regex[0]
        longitude =  regex[1]
        geolocate_tuple =(latitude,longitude,location)
        return geolocate_tuple
    except:
        print("Error")

In [690]:
geolocate('TSLA')

(':37.394738', '-122.149112', '3500 Deer Creek Road, Palo Alto, CA, 94304')

In [693]:
geolocate('WMT')

('36.364272', '-94.216793', '702 SW 8th Street, Bentonville, AR, 72716')

## Question 4

Now let's say we want to calculate the distance between two corporate headquarters.

Given two pairs of latitude and longitude, $(\varphi_1, \lambda_1)$ and $(\varphi_2, \lambda_2)$, we can calculate the distance between them using the [Haversine formula](https://en.wikipedia.org/wiki/Haversine_formula#The_haversine_formula):

$${\displaystyle d=2r\arcsin \left({\sqrt {\operatorname {hav} (\varphi _{2}-\varphi _{1})+\cos(\varphi _{1})\cos(\varphi _{2})\operatorname {hav} (\lambda _{2}-\lambda _{1})}}\right)}$$

where $d$ is the distance, $\operatorname{hav}$ is the Haversine function, $$\operatorname {hav} (\theta )=\sin ^{2}\left({\frac {\theta }{2}}\right),$$ and $r$ is the radius of the earth ([3958.7613 miles](https://en.wikipedia.org/wiki/Earth_radius#Mean_radius)). Note that this formula requires that the latitude and longitude are given in *radians*, not the degrees that Mapquest gives.

Use the template below to write a function that takes two ticker symbols and returns the distance between the headquarters of the two firms. Note that the python function for $\arcsin$ is `asin`.

In [705]:
def haversine(theta):
    '''Haversine function'''
    from math import sin
    return sin(theta/2)**2

def firm_dist(ticker1, ticker2):
    '''Returns the distance in miles between two firm HQs'''  
    from math import radians, cos, sin, asin, sqrt
    Earth_radius = 3958.7613
    # get lat/lng of two tickers
    ticker1_lat,ticker1_long,location1 = geolocate(ticker1)
    ticker2_lat,ticker2_long,location2 = geolocate(ticker2)
    # change degrees to radians
    ticker1_lat_rad = radians(float(ticker1_lat))
    ticker2_lat_rad = radians(float(ticker2_lat))
    ticker1_long_rad = radians(float(ticker1_long))
    ticker2_long_rad = radians(float(ticker2_long))    
    # calculate distance
    inner_formula = haversine(ticker1_lat_rad - ticker2_lat_rad) + cos(ticker1_lat_rad) * cos(ticker2_lat_rad) * haversine(ticker1_long_rad - ticker2_long_rad)
    sqrt_inner_formula =sqrt(inner_formula)
    d = 2 * asin(sqrt_inner_formula)  
    print(d)
    return d

In [706]:
# test your function
d = firm_dist('GOOG', 'FB')


0.0013711613644450992


In [707]:
d

0.0013711613644450992

In [708]:
print(f'Google and Facebook are {d:.1f} miles apart')

Google and Facebook are 0.0 miles apart
